### Can we classify each phase as relatively low or high anxiety for each subject? ###
#### APD, WESAD ####

In [4]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import shap
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    # train.Metrics.HF_RR, 
    # train.Metrics.LF_RR, 
    # train.Metrics.IBI, 
    # train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

model_phases_apd = [
    "Baseline_Rest", 
    "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
    "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
]

model_phases_wesad = dr_w.Phases.PHASE_ORDER

# anxiety_label_type = "Anxiety"
anxiety_label_type = None
wesad_label_type = "stai"

models = {
    # "SVM": SVC(C=10, gamma=1),  # C=10, gamma=1
    # "KNN": KNeighborsClassifier(n_neighbors=7),
    # "DT": DecisionTreeClassifier(),
    "LogReg": LogisticRegression(max_iter=1000, solver="liblinear"),
    # "Bayes": GaussianNB(),
    "RF": RandomForestClassifier(n_estimators=50, max_features=3),
    "XGB": XGBClassifier(use_label_encoder=False, objective="binary:logistic", eval_metric="error")
}

threshold = "fixed"
test_size = 1.0

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

temp_a, _ = train.Train_WESAD.get_wesad_data([train.Metrics.BPM], phases=dr_w.Phases.PHASE_ORDER, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_wesad_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_wesad_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

In [6]:
# TRAIN ON APD AND TEST ON WESAD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

if anxiety_label_type is not None:
    x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=False, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"] + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

y_train:
0    232
1    173
Name: label, dtype: int64
y_test:
0    55
1    15
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [64  6]
Model RF, Predictions: [0 1], [14 56]
Model XGB, Predictions: [0 1], [14 56]

[('mean_SCL', 2.0114337519686365), ('bpm', 0.3550269424353637), ('rmssd', -0.10214861201671672), ('SCR_rate', -0.841911806497848)]


[('mean_SCL', 0.3113560909439366), ('bpm', 0.25262094227255644), ('rmssd', 0.222138376560804), ('SCR_rate', 0.213884590222703)]


[('mean_SCL', 0.32396048), ('bpm', 0.2378228), ('rmssd', 0.2308749), ('SCR_rate', 0.20734185)]

Model evaluation metrics for LogReg:
	Accuracy: 0.8428571428571429
	Precision: 0.8333333333333334
	Recall: 0.3333333333333333
	F1-score: 0.47619047619047616
	AUC score: 0.6575757575757576
----------------------------------------

Model evaluation metrics for RF:
	Accuracy: 0.32857142857142857
	Precision: 0.21428571428571427
	Recall: 0.8
	F1-score: 0.3380281690140845
	AUC score: 0.5
---------------------------------

In [7]:
# TRAIN ON APD AND TEST ON WESAD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


# for j, phases_wesad in enumerate(model_phases_wesad):
for j, phases_wesad in enumerate([dr_w.Phases.TSST]):
    phases_wesad = [phases_wesad]
    print(f"WESAD PHASES {phases_wesad} " + "-"*50)
    x_a, y_a = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
    x_b, y_b = train.Train_WESAD.get_wesad_data(metrics, phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    x_a = x_a[~x_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    y_a = y_a[~y_a["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)

    if anxiety_label_type is not None:
        x_a = x_a.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, save_metrics=True, test_size=test_size, is_resample=True, get_importance=get_importance, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        print(f"Model evaluation metrics for {model_name}:")
        for i in range(len(reports[model_name])):
            report = reports[model_name][i]
            acc = acc_results[model_name][i]
            p = report["precision"]
            r = report["recall"]
            f1 = report["f1"]
            auc = report["auc"]
            print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
        print("")
    print("\n")

WESAD PHASES ['TSST'] --------------------------------------------------
Ratio of negative to positive labels (0.07692307692307693) is under 0.333, oversampling negative class.
y_train:
0    232
1    173
Name: label, dtype: int64
y_test:
1    13
0     4
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [11  6]
Model RF, Predictions: [0 1], [ 1 16]
Model XGB, Predictions: [0 1], [ 2 15]

[('mean_SCL', 2.0114337519686365), ('bpm', 0.3550269424353637), ('rmssd', -0.10214861201671672), ('SCR_rate', -0.841911806497848)]


[('mean_SCL', 0.30878891852644574), ('bpm', 0.2434446313252888), ('rmssd', 0.23164954732195128), ('SCR_rate', 0.2161169028263142)]


[('mean_SCL', 0.32396048), ('bpm', 0.2378228), ('rmssd', 0.2308749), ('SCR_rate', 0.20734185)]

Model evaluation metrics for LogReg:
	Accuracy: 0.5882352941176471
	Precision: 1.0
	Recall: 0.46153846153846156
	F1-score: 0.631578947368421
	AUC score: 0.7307692307692308
----------------------------------------

Model evaluation metrics

In [8]:
# TRAIN ON WESAD AND TEST ON APD -- ALL
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
# x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, model_phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
# drop subjects with noisy data
x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
if anxiety_label_type is not None:
    x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

x_a = x_a.drop(["phaseId"], axis=1)
x_b = x_b.drop(["phaseId"], axis=1)

# make sure subjects from different datasets aren't labeled with the same index
x_b["subject"] = x_b["subject"] + 500
y_b["subject"] = y_b["subject"] + 500

acc_results = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
reports = {
    # "SVM": [],
    "LogReg": [],
    "RF": [],
    "XGB": []
}
num_iters = 1
get_importance = True
for _ in range(num_iters):
    out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, is_resample=True, test_size=test_size, save_metrics=True, get_importance=get_importance, drop_subject=True)
    for model_name in acc_results:
        acc_results[model_name].append(out[model_name][0])
        reports[model_name].append(out[model_name][1])
        if get_importance:
            try:
                print("")
                feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                print(feature_imp)
            except Exception as e:
                print(e)
                # print(out[model_name][0][2])
            print("")

for model_name in acc_results.keys():
    print(f"Model evaluation metrics for {model_name}:")
    for i in range(len(reports[model_name])):
        report = reports[model_name][i]
        acc = acc_results[model_name][i]
        p = report["precision"]
        r = report["recall"]
        f1 = report["f1"]
        auc = report["auc"]
        print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
    print("")
print("\n")

Ratio of positive to negative labels (0.2727272727272727) is under 0.333, oversampling positive class.
y_train:
0    55
1    18
Name: label, dtype: int64
y_test:
0    232
1    173
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [390  15]
Model RF, Predictions: [0 1], [236 169]
Model XGB, Predictions: [0 1], [297 108]

[('bpm', 1.9852045677022874), ('mean_SCL', 1.5304846712662117), ('SCR_rate', 0.6281990449459239), ('rmssd', -0.29650526184561926)]


[('bpm', 0.41953361443318377), ('mean_SCL', 0.3552936424153309), ('SCR_rate', 0.11894613286553879), ('rmssd', 0.10622661028594652)]


[('bpm', 0.40713423), ('mean_SCL', 0.32660735), ('SCR_rate', 0.17054918), ('rmssd', 0.095709264)]

Model evaluation metrics for LogReg:
	Accuracy: 0.5851851851851851
	Precision: 0.6666666666666666
	Recall: 0.057803468208092484
	F1-score: 0.10638297872340426
	AUC score: 0.5181258720350808
----------------------------------------

Model evaluation metrics for RF:
	Accuracy: 0.5555555555555556
	Precis

In [9]:
# TRAIN ON WESAD AND TEST ON APD
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dt)


# for j, phases_apd in enumerate(model_phases_apd):
for j, phases_apd in enumerate([dr_a.Phases.BUG_EXPOSURE, dr_a.Phases.SPEECH_EXPOSURE]):
    print(f"APD PHASES {phases_apd} " + "-"*50)
    phases_apd = [phases_apd]
    x_a, y_a = train.Train_WESAD.get_wesad_data(metrics, model_phases_wesad, verbose=False, label_type=wesad_label_type, threshold=threshold, normalize=True)
    x_b, y_b = train.Train_APD.get_apd_data_ranking(metrics, phases_apd, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
    # drop subjects with noisy data
    x_b = x_b[~x_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    y_b = y_b[~y_b["subject"].isin(invalid_apd_subjects)].reset_index(drop=True)
    if anxiety_label_type is not None:
        x_b = x_b.drop(["anxietyGroup"], axis=1)  # drop anxietyGroup column because WESAD doesn't have this feature

    x_a = x_a.drop(["phaseId"], axis=1)
    x_b = x_b.drop(["phaseId"], axis=1)

    # make sure subjects from different datasets aren't labeled with the same index
    x_b["subject"] = x_b["subject"] + 500
    y_b["subject"] = y_b["subject"] + 500

    acc_results = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    reports = {
        # "SVM": [],
        "LogReg": [],
        "RF": [],
        "XGB": []
    }
    num_iters = 1
    get_importance = True
    for _ in range(num_iters):
        out = train.Train_Multi_Dataset.train_across_datasets(models, x_a, y_a, x_b, y_b, by_subject=True, is_resample=True, test_size=test_size, save_metrics=True, get_importance=get_importance, drop_subject=True)
        for model_name in acc_results:
            acc_results[model_name].append(out[model_name][0])
            reports[model_name].append(out[model_name][1])
            if get_importance:
                try:
                    print("")
                    feature_imp = list(zip(metrics + ["lf_hf_ratio"], out[model_name][2]))
                    feature_imp = sorted(feature_imp, key=lambda x: x[1], reverse=True)
                    print(feature_imp)
                except Exception as e:
                    print(e)
                    # print(out[model_name][0][2])
                print("")

    for model_name in acc_results.keys():
        print(f"Model evaluation metrics for {model_name}:")
        for i in range(len(reports[model_name])):
            report = reports[model_name][i]
            acc = acc_results[model_name][i]
            p = report["precision"]
            r = report["recall"]
            f1 = report["f1"]
            auc = report["auc"]
            print(f"\tAccuracy: {acc}\n\tPrecision: {p}\n\tRecall: {r}\n\tF1-score: {f1}\n\tAUC score: {auc}\n" + "-"*40)
        print("")
    print("\n")

APD PHASES BugBox_Exposure --------------------------------------------------
Ratio of positive to negative labels (0.2727272727272727) is under 0.333, oversampling positive class.
Ratio of negative to positive labels (0.21621621621621623) is under 0.333, oversampling negative class.
y_train:
0    55
1    18
Name: label, dtype: int64
y_test:
1    37
0    12
Name: label, dtype: int64
Model LogReg, Predictions: [0 1], [43  6]
Model RF, Predictions: [0 1], [24 25]
Model XGB, Predictions: [0 1], [36 13]

[('bpm', 2.0905170386138123), ('mean_SCL', 1.3616212341081455), ('SCR_rate', 0.6291135440956427), ('rmssd', -0.3276856701712501)]


[('bpm', 0.4563276313471523), ('mean_SCL', 0.38568317962279886), ('rmssd', 0.08202586469521006), ('SCR_rate', 0.07596332433483878)]


[('bpm', 0.41343448), ('mean_SCL', 0.3405777), ('SCR_rate', 0.15725216), ('rmssd', 0.0887357)]

Model evaluation metrics for LogReg:
	Accuracy: 0.3673469387755102
	Precision: 1.0
	Recall: 0.16216216216216217
	F1-score: 0.2790697